In [1]:
#import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [2]:

df = pd.read_csv("/content/drug200.csv")
df.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY


In [3]:
#separating features from labels
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [4]:
#Label encoding to convert string to numerical values
le = LabelEncoder()
X[:, 1] = le.fit_transform(X[:, 1])


ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [2, 3])], remainder='passthrough')
X = np.array(ct.fit_transform(X))


OH = OneHotEncoder()
encoded_y = OH.fit(y.reshape(-1,1))
y = encoded_y.transform(y.reshape(-1,1)).toarray()

In [5]:
X[0],y[0]

(array([1.0, 0.0, 0.0, 1.0, 0.0, 23, 0, 25.355], dtype=object),
 array([1., 0., 0., 0., 0.]))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [18]:
len(X_train),len(X_test)

(160, 40)

In [7]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=15, activation='relu'))
ann.add(tf.keras.layers.Dense(units=10, activation='relu'))
ann.add(tf.keras.layers.Dense(units=5, activation='relu'))
outputs =  len(set(df["Drug"]))
ann.add(tf.keras.layers.Dense(units=outputs, activation='softmax'))

In [9]:
ann.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
ann.fit(X_train, y_train, batch_size=16, epochs=100)

Epoch 1/100
10/10 [==============================] - 2s 12ms/step - loss: 1.7858 - accuracy: 0.1063
Epoch 2/100
10/10 [==============================] - 0s 7ms/step - loss: 1.7048 - accuracy: 0.1375
Epoch 3/100
10/10 [==============================] - 0s 9ms/step - loss: 1.6533 - accuracy: 0.1937
Epoch 4/100
10/10 [==============================] - 0s 5ms/step - loss: 1.6164 - accuracy: 0.2750
Epoch 5/100
10/10 [==============================] - 0s 8ms/step - loss: 1.5874 - accuracy: 0.3313
Epoch 6/100
10/10 [==============================] - 0s 7ms/step - loss: 1.5648 - accuracy: 0.4062
Epoch 7/100
10/10 [==============================] - 0s 7ms/step - loss: 1.5457 - accuracy: 0.4750
Epoch 8/100
10/10 [==============================] - 0s 9ms/step - loss: 1.5269 - accuracy: 0.5312
Epoch 9/100
10/10 [==============================] - 0s 8ms/step - loss: 1.5091 - accuracy: 0.5500
Epoch 10/100
10/10 [==============================] - 0s 6ms/step - loss: 1.4920 - accuracy: 0.5750
Epoch 11

In [11]:
y_pred = ann.predict(X_test)

2/2 [==============================] - 1s 7ms/step


In [12]:
y_pred_outputs = []
for output in y_pred:
    output_list = [0] * outputs
    output_list[np.where(output == max(output))[0][0]] = 1
    y_pred_outputs.append(output_list)

In [16]:
y_pred_outputs[0]

[1, 0, 0, 0, 0]

In [17]:


accuracy_score(y_test, y_pred_outputs)




0.975